In [ ]:
import json
from experiment.Experiment import Experiment
from account_creation.GoogleWorkspace import GoogleWorkspace
from utils import shuffleIP as IP
from utils.log import debug, error
from utils.util import wait, bigWait

config = json.load(open('config.json', 'r'))
platforms = config['platforms']
print(config)

In [ ]:
GW = GoogleWorkspace()
if GW.needUsers(config):
    GW.createUsers(config)

In [ ]:
experiment_subjects = dict()
for platform in platforms:
    experiment = Experiment('config.json', platform)
    experiment.initiate()
    experiment_subjects[platform] = experiment.get_subjects()

subjects = dict()
for platform in platforms:
    for subject in experiment_subjects[platform]:
        if subject.id not in subjects:
            subjects[subject.id] = dict()
        subjects[subject.id][platform] = subject

# Initate

In [ ]:
subject_names = list(subjects.keys())
for subject_name in subject_names:
    IP.shuffle(subject_name)
    for platform in platforms:
        print(f'Platform: {platform}, Subject: {subject_name}')
        subject = subjects[subject_name][platform]
        signed = subject.platformSignin()
        debug(f'Platform: {platform}, Subject: {subject_name}, Signed: {signed}')
        input('Press Enter to continue')

In [ ]:
subject_names

# Observation

In [ ]:
subject_names = list(subjects.keys())
for subject_name in subject_names:
    IP.shuffle(subject_name)
    chrome = subjects[subject_name]['YouTube']
    signed = chrome.checkChromeSignin()
    debug(f'Chrome signed in: {signed}')
    if not signed:
        error(f'Chrome not signed in: {subject_name}')
        chrome.chromeSignIn()

    for platform in platforms:
        print(f'Platform: {platform}, Subject: {subject_name}')
        subject = subjects[subject_name][platform]
        debug(f'Checking signin for {subject_name} on {platform}')
        wait(3)
        if not subject.checkSignin():
            error(f'{subject_name} not signed in on {platform}')
            error(f'Attempting to sign in {subject_name} on {platform}')
            wait(3)
            try:
                subject.platformSignIn()
            except Exception as e:
                error(f'Error signing in {subject_name} on {platform}')
                error(e)
                continue
            wait(3)

        try:
            debug(f'Observing {subject_name} on {platform}')
            subject.observe(pre=True)
        except Exception as e:
            error(f'Error observing {subject_name} on {platform}')
            error(e)
    bigWait(10)

# Treatment

In [5]:
import os
def closeAllChrome():
    try:
        os.system("taskkill /im chrome.exe /f")
    except Exception as e:
        error(e)

In [4]:
subject_names = list(subjects.keys())
for subject_name in subject_names[1:]:
    IP.shuffle(subject_name)
    chrome = subjects[subject_name]['YouTube']
    signed = chrome.checkChromeSignin()
    debug(f'Chrome signed in: {signed}')
    if not signed:
        error(f'Chrome not signed in: {subject_name}')
        chrome.chromeSignIn()

    for platform in platforms:
        print(f'Platform: {platform}, Subject: {subject_name}')
        subject = subjects[subject_name][platform]
        debug(f'Checking signin for {subject_name} on {platform}')
        wait(3)
        if not subject.checkSignin():
            error(f'{subject_name} not signed in on {platform}')
            error(f'Attempting to sign in {subject_name} on {platform}')
            wait(3)
            try:
                subject.platformSignIn()
            except Exception as e:
                subject.close()
                error(f'Error signing in {subject_name} on {platform}')
                error(e)
                continue
            wait(3)

        try:
            debug(f'Treatment {subject_name} on {platform}')
            subject.treatment()
        except Exception as e:
            error(f'Error observing {subject_name} on {platform}')
            error(e)
    closeAllChrome()
    bigWait(10)

shuffle:	 Shuffling IP for like_oscars_1
setIP:	 Setting New IP: : 128.255.45.121
getCurrentIP:	 Current IP Address: 128.255.45.115
getCurrentIP:	 Current IP Address: 128.255.45.115


KeyboardInterrupt: 

In [ ]:
bigWait(30)
subject_names = list(subjects.keys())
for subject_name in subject_names:
    IP.shuffle(subject_name)
    chrome = subjects[subject_name]['YouTube']
    signed = chrome.checkChromeSignin()
    debug(f'Chrome signed in: {signed}')
    if not signed:
        error(f'Chrome not signed in: {subject_name}')
        chrome.chromeSignIn()

    for platform in platforms:
        print(f'Platform: {platform}, Subject: {subject_name}')
        subject = subjects[subject_name][platform]
        debug(f'Checking signin for {subject_name} on {platform}')
        wait(3)
        if not subject.checkSignin():
            error(f'{subject_name} not signed in on {platform}')
            error(f'Attempting to sign in {subject_name} on {platform}')
            wait(3)
            try:
                subject.platformSignIn()
            except Exception as e:
                error(f'Error signing in {subject_name} on {platform}')
                error(e)
                continue
            wait(3)

        try:
            debug(f'Observing {subject_name} on {platform}')
            subject.observe(pre=True)
        except Exception as e:
            error(f'Error observing {subject_name} on {platform}')
            error(e)
    bigWait(10)

In [ ]:
treatments = dict()
subject_names = list(subjects.keys())
for subject_name in subject_names:
    treatments[subject_name] = dict()
    for platform in subjects[subject_name]:
        try:
            treatments[subject_name][platform] = subject.get_treatments()
        except Exception as e:
            error(e)

In [ ]:
import pickle as pkl
import pandas as pd
import json
import sqlite3 as sql

from experiment.Experiment import Experiment
from experiment.Subject import Subject

def uploadResults(platform):
    config_file = 'config.json'
    config = json.load(open(config_file, 'r'))
    experiment = Experiment(config_file, platform)
    subjects = experiment.get_subjects()

    users = []
    for user in subjects:
        treatments = user.get_treatments()
        for tick in treatments:
            pre, post = user.get_observations(tick)
            treatments[tick]['pre'] = pre
            treatments[tick]['post'] = post

        users.append(treatments)

    pkl.dump(users, open(f'{platform}-results.pkl', 'wb'))

    return users